In [ ]:
from constants import img_size
import numpy as np
from keras import backend as K
from keras.layers import Input, Dense, Flatten, BatchNormalization, subtract
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.regularizers import l2
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook
from IPython.display import clear_output

## Model

In [ ]:
class Seamese_Model:
    def __init__(self, base_model, opts, kernel_opts=None, bias_opts=None):
        self.base_model = base_model
        self.opts = opts
        self.kernel_opts = kernel_opts
        self.bias_opts = bias_opts
    
    def kernel_initializer(self, shape, name=None):
        if (self.kernel_opts != None):
            values = np.random.normal(loc=self.kernel_opts['loc'], scale=self.kernel_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
        
    def bias_initializer(self, shape, name=None):
        if (self.bias_opts != None):
            values = np.random.normal(loc=self.bias_opts['loc'], scale=self.bias_opts['scale'], size=shape)
            return K.variable(values, name=name)
        else:
            return None
    
    def get_top_model(self):
        input_1 = Input(self.opts['features_shape'])
        input_2 = Input(self.opts['features_shape'])
        
        X_1 = Flatten()(input_1)
        X_2 = Flatten()(input_2)
        X = subtract([X_1, X_2])
        X = Dense(self.opts['features_shape'][1], activation='relu', name='dense_0', kernel_regularizer=l2(1e-3))(X)
        X = BatchNormalization()(X)
        output = Dense(1, activation='sigmoid', name='log_reg', kernel_regularizer=l2(1e-3))(X)
        
        model = Model(input=[input_1, input_2], output=output)
        return model

## Supervisor

In [ ]:
class Supervisor:
    # use seed only for testing
    def __init__(self, model, data_train, classes_train, filenames_train, data_dev, classes_dev, filenames_dev, seed=None):
        self.model = model
        self.data_train = data_train
        self.classes_train = classes_train
        self.filenames_train = filenames_train
        self.data_dev = data_dev
        self.classes_dev = classes_dev
        self.filenames_dev = filenames_dev
        self.seed = seed
        
    def get_pair(self, index_1, index_2):
        el_1 = np.take(self.data_train, [index_1], axis=0)
        el_2 = np.take(self.data_train, [index_2], axis=0)
        return([el_1, el_2])
    
    def get_selection_index(self, index, indices):
        selection_index = index
        while selection_index == index:
            selection_index = np.random.choice(indices, 1)[0]
        return selection_index
    
    def get_batch(self, n, data, classes):
        np.random.seed(self.seed)
        indices = np.random.choice(list(range(len(data))), size=n)
        pairs = []
        y = []
        
        for index in indices[:n//2]:
            selection_indices = np.argwhere(classes == classes[index]).flatten()
            selection_index = self.get_selection_index(index, selection_indices)
            pairs.append(self.get_pair(index, selection_index))
            y.append(1)
            
        for index in indices[n//2:]:
            selection_indices = np.argwhere(classes != classes[index]).flatten()
            selection_index = self.get_selection_index(index, selection_indices)
            pairs.append(self.get_pair(index, selection_index))
            y.append(0)
            
        return (np.array(pairs), np.array(y))
    
    def train(self, iterations, batch_size, validation_size=0, validate_every=float('inf'), learning_rate=0.0001, path=None, k=[1]):
        self.model.compile(optimizer=tf.train.AdamOptimizer(learning_rate),
                  loss='binary_crossentropy',
                  metrics=['acc'])
        
        if path != None:
            self.model.load_weights(path)
            
        for i in tqdm_notebook(range(iterations)):
            inputs, targets = self.get_batch(batch_size, self.data_train, self.classes_train)
            metrics = self.model.train_on_batch([inputs[:,0,:], inputs[:,1,:]], targets)
            if (i % validate_every == 0) & (i != 0):
                self.checkpoint(batch_size, validation_size, i, metrics, k)
    
    def get_validation_task(self, data, classes, filenames, show_filenames=False, index = None):
        if index == None:
            index = np.random.choice(range(data.shape[0]), 1)[0]
        
        targets = np.repeat(np.take(data, [index], axis=0), data.shape[0] - 1, axis=0)
        support_set = np.delete(data, index, axis=0)
        pairs = np.stack([targets, support_set], axis=1)
        
        pair_filenames = None
        pair_classes = None
        target_class = None
        support_classes = None
        if show_filenames == True:
            target_filenames = np.repeat(np.take(filenames, [index], axis=0), data.shape[0] - 1, axis=0)
            target_class = np.take(classes, [index], axis=0)
            support_filenames = np.delete(filenames, index, axis=0)
            support_classes = np.delete(classes, index, axis=0)
            pair_filenames = np.stack([target_filenames, support_filenames], axis=1)
        
        target_y = classes[index]
        y = (np.delete(classes, index) - target_y) == 0

        return pairs.reshape(pairs.shape[0], pairs.shape[1], 1, pairs.shape[2]), y, pair_filenames, target_class, support_classes
    
    def calculate_accuracy(self, n, batch_size, data, classes, filenames, ks, verbose=False, show_filenames=False):
        incorrect_answers_count = np.zeros(len(ks))
        incorrect_filenames = np.zeros(len(ks))
        
        for i in tqdm_notebook(range(n)):
            inputs, targets, pair_filenames, target_class, support_classes = self.get_validation_task(data, classes, filenames, show_filenames, i)
            predictions = self.model.predict([inputs[:,0,:], inputs[:,1,:]], batch_size=batch_size)
            
            for ind, k in enumerate(ks):
                probs = np.flipud(np.sort(predictions.flatten()))[0:k]
                indices = np.flipud(np.argsort(predictions.flatten()))[0:k]
                pred_classes = np.take(support_classes, indices, axis=0)
                unique_classes = np.unique(pred_classes)

                class_to_index = {}
                for i, cl in enumerate(unique_classes):
                    class_to_index[cl] = i

                counts = np.zeros(len(unique_classes))
                sum_probs = np.zeros(len(unique_classes))
                for cl, prob in zip(pred_classes, probs):
                    counts[class_to_index[cl]] += 1
                    sum_probs[class_to_index[cl]] += prob

                pred_class = None
                max_count = 0
                max_sum_prob = 0
                for cl, count, sum_prob in zip(unique_classes, counts, sum_probs):
                    if count > max_count:
                        pred_class = cl
                        max_count = count
                        max_sum_prob = sum_prob
                    elif (count == max_count) & (sum_prob > max_sum_prob):
                        pred_class = cl
                        max_count = count
                        max_sum_prob = sum_prob

                if pred_class != target_class[0]:
                    incorrect_answers_count[ind] += 1
#                if show_filenames == True:
#                    incorrect_filenames.append(pair_filenames[index])
                
        return list(map(lambda x: 1 - x/n, incorrect_answers_count))
    
    def checkpoint(self, batch_size, validation_size, iteration, metrics, ks=[1]):
#        train_acc, incorrect_filenames_train = self.calculate_accuracy(
#            n=validation_size, 
#            batch_size=batch_size, 
#            data=self.data_train, 
#            classes=self.classes_train, 
#            filenames=self.filenames_train, 
#            ks=ks,
#        )
        dev_acc = self.calculate_accuracy(
            n=validation_size, 
            batch_size=batch_size, 
            data=self.data_dev, 
            classes=self.classes_dev, 
            filenames=self.filenames_dev, 
            ks=ks,
            show_filenames=True,
        )
        print(dev_acc)
        print('Iteration ' + str(iteration) + '. Batch metrics [loss, accuracy]:', metrics)
        for i, k in enumerate(ks):
            print('Iteration ' + str(iteration) + '. Validation accuracy (k=' + str(k) + '):', dev_acc[i])
        print('---------')
        self.model.save_weights('../input/siamese_model_weights/weights-' + str(iteration) + '-' + str(round(dev_acc[0], 4)) + '.hdf5')
        
    def validate(self, weights, data, classes, filenames, n=350, batch_size=128, ks=[5], verbose=False):
        self.model.load_weights(weights)
        
        acc = self.calculate_accuracy(
            n=n, 
            batch_size=batch_size, 
            data=data, 
            classes=classes, 
            filenames=filenames, 
            ks=ks,
            show_filenames=True,
            verbose=verbose,
        )
        
        return acc

In [ ]:
features_train = np.load('../input/features/siamese/features_train.npy')
features_dev = np.load('../input/features/siamese/features_dev.npy')
classes_train = np.load('../input/features/siamese/classes_train.npy')
classes_dev = np.load('../input/features/siamese/classes_dev.npy')
filenames_train = np.load('../input/features/siamese/filenames_train.npy')
filenames_dev = np.load('../input/features/siamese/filenames_dev.npy')

In [ ]:
model = Seamese_Model(
    ResNet50, 
    opts={
        'features_shape': (1, features_train.shape[1]),
    },
)

model = model.get_top_model()

In [ ]:
model.summary()

In [ ]:
supervisor = Supervisor(
    model=model, 
    data_train=features_train, 
    classes_train=classes_train, 
    filenames_train=filenames_train,
    data_dev=features_dev, 
    classes_dev=classes_dev,
    filenames_dev=filenames_dev,
)

## Training

In [ ]:
supervisor.train(
    iterations=250000, 
    batch_size=128, 
    validation_size=350, 
    validate_every=5000, 
    learning_rate=0.00001,
    k=[1,5,15,20],
    #path='./siamese_model_weights/weights-55000-0.4714.hdf5',
 )

## Validation

In [ ]:
from scipy import ndimage
from matplotlib import pyplot as plt

In [ ]:
acc = supervisor.validate(
    weights='../input/siamese_model_weights/weights-30000-0.8257.hdf5', 
    data=features_dev, 
    classes=classes_dev, 
    filenames=filenames_dev, 
    n=features_dev.shape[0], 
    batch_size=128, 
    ks=[1,5,15,20],
    verbose=False,
)
print(acc)